In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-09-23"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
583,2024-09-23,Paraguai Lnb,20:30,Olimpia,Dep. San Jose,2.01,1.68,147.5,1.85,1.85,1.5,1.88,1.78,rcsXOj5U,0.497512,0.595238,0.540541,0.540541,0.092751,181.546,113.389934,0.624580,1.2,1.643168,1.369306,381.48,2.692,1.708192,0.634544,-7.0,99.012,15.078500,0.152290,3.0,0.000000,0.000000,79.57,1.234,0.262164,0.212451,94.0,68,73,5.61,1.09,119.654,497.506,0.126474,0.000000,0.038640,-0.41,-0.082,-12.317073,0.00000,0.0,0.00000,21.89,4.378,0.155322,0.0,0.0,0.0
584,2024-09-23,Romênia Divisão A,13:00,CSU Sibiu,Brasov,5.72,1.12,162.5,1.83,1.83,7.5,2.00,1.69,xpMkr0BL,0.174825,0.892857,0.546448,0.546448,0.067682,194.232,86.307571,0.444353,0.6,1.341641,2.236068,187.50,2.274,1.107217,0.486903,-38.0,229.032,55.872162,0.243949,0.6,1.341641,2.236068,231.84,2.816,0.797766,0.283297,-21.0,75,84,2.50,2.76,153.086,288.828,0.951079,0.000000,0.118809,-1.34,-0.268,-17.611940,0.53256,0.5,-0.03256,-2.07,-0.414,-0.289855,0.0,0.0,0.0
585,2024-09-23,Paraguai Lnb,20:30,Felix Perez Cardozo,Ciudad Nueva,1.02,8.80,144.5,1.83,1.83,-19.5,1.95,1.72,jsTNSY55,0.980392,0.113636,0.546448,0.546448,0.094029,239.846,190.387826,0.793792,1.2,1.643168,1.369306,86.94,3.564,3.408142,0.956269,-16.0,324.688,289.709792,0.892271,1.2,1.643168,1.369306,227.80,5.548,5.451153,0.982544,-20.0,69,68,1.26,3.35,156.542,0.000,1.120426,0.000000,0.088629,-2.06,-0.412,-0.048544,0.50132,0.5,-0.00132,0.00,0.000,inf,0.0,0.0,0.0
586,2024-09-23,Turquia Tbl,13:00,Bordo Sportif,Balikesir,1.61,2.12,151.5,1.84,1.82,-3.5,1.90,1.76,zX3oV2eC,0.621118,0.471698,0.543478,0.549451,0.092816,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,0,0.00,0.00,0.000,0.000,0.193364,0.007728,0.054096,0.00,0.000,inf,0.00000,0.0,0.00000,0.00,0.000,inf,0.0,0.0,0.0
587,2024-09-23,Filipinas Copa Dos Governadores,08:30,Rain Or Shine Paint.,Blackwater Bossing,1.35,3.09,223.5,1.85,1.85,-7.5,1.95,1.76,vZTfJiBE,0.740741,0.323625,0.540541,0.540541,0.064365,156.744,22.940128,0.146354,3.0,0.000000,0.000000,147.62,1.506,0.490031,0.325386,73.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,122,98,1.21,2.49,0.000,0.000,0.554219,0.000000,0.072426,0.00,0.000,inf,0.00000,0.0,0.00000,0.00,0.000,inf,0.0,0.0,0.0
588,2024-09-23,Filipinas Copa Dos Governadores,06:00,Magnolia Hotshots,Converge FiberXers,1.39,2.89,198.5,1.87,1.79,-6.5,1.91,1.80,bwi8elJ7,0.719424,0.346021,0.534759,0.558659,0.065445,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,124,96,1.20,3.77,0.000,0.000,0.495636,0.030912,0.041931,0.00,0.000,inf,0.00000,0.0,0.00000,0.00,0.000,inf,0.0,0.0,0.0
589,2024-09-23,Mundo Amistoso Interclubes,03:00,Ryukyu (Jpn),LG Sakers (Kor),1.18,4.47,160.5,1.85,1.85,-11.5,1.95,1.73,ID2J731E,0.847458,0.223714,0.540541,0.540541,0.071171,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,0,0.00,0.00,0.000,0.000,0.823498,0.000000,0.084545,0.00,0.000,inf,0.00000,0.0,0.00000,0.00,0.000,inf,0.0,0.0,0.0
590,2024-09-23,Israel Copa Da Liga,14:00,Hapoel Holon,Nes Ziona,1.19,3.93,161.5,1.82,1.84,-9.5,1.

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

Sem jogos
